<a href="https://colab.research.google.com/github/jmasonlee/efficiently_testing_etl_pipelines/blob/main/fixing_a_big_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## STEP 0.A: Setup Notebook

In [1]:
!rm -rf efficiently_testing_etl_pipelines
!git clone https://github.com/jmasonlee/efficiently_testing_etl_pipelines.git
!cp -r /content/efficiently_testing_etl_pipelines/src/ .
!cp -r /content/efficiently_testing_etl_pipelines/tests/ .
!rm -rf efficiently_testing_etl_pipelines
!rm -rf tests/diamond_pricing_test*
!rm -rf tests/test_helpers/*verification_helpers.py
!rm -rf tests/conftest.py
!rm -rf sample_data


Cloning into 'efficiently_testing_etl_pipelines'...
remote: Enumerating objects: 629, done.
remote: Counting objects: 100% (270/270), done.
remote: Compressing objects: 100% (193/193), done.
remote: Total 629 (delta 174), reused 111 (delta 70), pack-reused 359
Receiving objects: 100% (629/629), 271.00 KiB | 3.98 MiB/s, done.
Resolving deltas: 100% (380/380), done.


# STEP 0.B: Setup Tests

### Install Dependencies

For the exercise, we will need some special dependencies to allow us to run lots of tests in a notebook.

`ipytest` lets us run our tests in a notebook.



In [2]:
!pip install ipytest

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.8 MB/s eta 0:00:00


ipytest is what allows us to run our tests in a notebook. This next cell is not needed if you are writing tests in a separate pytest file.

In [3]:
import ipytest
ipytest.autoconfig()

We are installing `pyspark` because it doesn't come with the base colab environment

In [4]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=aeff8d6806c7acdd46694a55842c1f7d77cce8f67b6c965eaa6f6eab57fa0523
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [5]:
!pip install chispa

## Create a local SparkSession

Normally spark runs on a bunch of executors in the cloud. Since we want our tests to be able to run on a single dev machine, we make a fixture that gives us a local spark context.

In [6]:
import pytest
from _pytest.fixtures import FixtureRequest
from pyspark import SparkConf
from pyspark.sql import SparkSession

@pytest.fixture(scope="session")
def spark(request: FixtureRequest):
    conf = (SparkConf()
        .setMaster("local")
        .setAppName("sample_pyspark_testing_starter"))

    spark = SparkSession \
        .builder \
        .config(conf=conf) \
        .getOrCreate()

    request.addfinalizer(lambda: spark.stop())
    return spark

## Create Helpers

This is a helper function that retrieves our test output from the expected.json file

In [7]:
import json
from typing import List, Dict

from pyspark.pandas import DataFrame


def create_df_from_json(json_file, spark):
    return spark.read.option("multiline", "true").json(json_file)


def data_frame_to_json(df: DataFrame) -> List:
    output = [json.loads(item) for item in df.toJSON().collect()]
    output.sort(key=lambda item: item["id"])
    print(output)
    return output

def expected_json(name: str) -> Dict:
    with open(f"tests/fixtures/{name}") as f:
        return json.loads(f.read())

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [8]:
import pyspark
def build_indep_vars(df, independent_vars, categorical_vars=None, keep_intermediate=False, summarizer=True):
    check_input(categorical_vars, df, independent_vars)

    from pyspark.ml import Pipeline
    from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
    idx = 'index'
    vec = 'vector'
    if categorical_vars:
        string_indexer = [StringIndexer(inputCol=x,
                                        outputCol=f"{x}_{idx}")
                          for x in categorical_vars]

        encoder        = [OneHotEncoder(dropLast=True,
                                        inputCol =f'{x}_{idx}',
                                        outputCol=f'{x}_{vec}')
                          for x in categorical_vars]

        independent_vars = ['{}_vector'.format(x) if x in categorical_vars else x for x in independent_vars]
    else:
        string_indexer, encoder = [], []

    assembler = VectorAssembler(inputCols=independent_vars,
                                outputCol='indep_vars')
    pipeline  = Pipeline(stages=string_indexer+encoder+[assembler])
    model = pipeline.fit(df)
    df = model.transform(df)

    if not keep_intermediate:
        fcols = [c for c in df.columns if f'_{idx}' not in c[-3:] and f'_{vec}' not in c[-7:]]
        df = df[fcols]

    return df


def check_input(categorical_vars, df, independent_vars):
    assert (type(
        df) is pyspark.sql.dataframe.DataFrame), 'pypark_glm: A pySpark dataframe is required as the first argument.'
    assert (type(
        independent_vars) is list), 'pyspark_glm: List of independent variable column names must be the third argument.'
    for iv in independent_vars:
        assert (type(iv) is str), 'pyspark_glm: Independent variables must be column name strings.'
        assert (iv in df.columns), 'pyspark_glm: Independent variable name is not a dataframe column.'
    if categorical_vars:
        for cv in categorical_vars:
            assert (type(cv) is str), 'pyspark_glm: Categorical variables must be column name strings.'
            assert (cv in df.columns), 'pyspark_glm: Categorical variable name is not a dataframe column.'
            assert (cv in independent_vars), 'pyspark_glm: Categorical variables must be independent variables.'


In [9]:
from pyspark.sql import DataFrame, Window, Column
from pyspark.sql.functions import log, when, mean, col

def replace_null(orig: Column, average: Column):
    return when(orig.isNull(), average).otherwise(orig)

def transform(df: DataFrame) -> DataFrame:

    df = df.withColumn('lprice', log('price'))
    window = Window.partitionBy('cut', 'clarity').orderBy('price').rowsBetween(-3, 3)
    moving_avg = mean(df['price']).over(window)
    df = df.withColumn('moving_avg', moving_avg)

    df = df.withColumn('price', when(df.price.isNull(), df.moving_avg).otherwise(df.price))
    df = df[['id', 'carat', 'clarity', 'color', 'price']]
    df = build_indep_vars(df, ['carat', 'clarity', 'color'],
                                      categorical_vars=['clarity', 'color'],
                                      keep_intermediate=False,
                                      summarizer=True)
    return df

# STEP 0.C: Run The Test

In [ ]:
%%ipytest -qq
from pyspark.sql import SparkSession

def test_will_do_the_right_thing(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    actual_df = transform(diamonds_df)
    assert data_frame_to_json(actual_df) == expected_json("expected.json")

.                                                                                            [100%]


# Step 1: Setup For the Saff Squeeze

## Instructions
Let's get ready to improve the test.

**our bug**: Diamonds of the same cut and clarity are influencing the calculated price of diamonds with a different color. Only diamonds with the same cut, clarity _**and color**_ should be influencing the calculated price for diamonds with a null price.

**expected behaviour**:
An unpriced diamond with cut=Good, color=D and clarity=VVS1 in a dataset with other diamonds of the same cut, clarity and color all priced at 2690.0, will have it's price set to match the average of all the prices for diamonds of the same cut, clarity and color - or 2690.0.



- [ ] Change the test to check for the behaviour we want.
There is a second json file (`expected_correct.json`) where the expected price for the unpriced diamond has been updated to the correct value. Use that file name as the argument passed to `expected_json`
- [ ] Run the test. It should fail.
- [ ] Duplicate the test. Now you should have two copies of the same test.
One copy will stay the same, so we can make sure that nothing is broken. The second copy is what we will change in the next steps.
- [ ] Rename the test.
Pick a name that tells you what behaviour you are verifying with the test you are using for the Saff Squeeze. I chose `test_null_price_is_replaced_based_on_cut_clarity_and_color`, but names are hard. You might be able to think of a better one :-)
- [ ] Run the tests. They should both fail because our diamond is being returned with a price of 2460.0, when we expect 2690.0

## Exercise

In [ ]:
%%ipytest -qq
from pyspark.sql import SparkSession

def test_will_do_the_right_thing(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    actual_df = transform(diamonds_df)
    assert data_frame_to_json(actual_df) == expected_json("expected.json")

.                                                                                            [100%]


# Step 2: Make The Assert Specific

Right now, our test compares everything in the output dataframe to everything in a large json file. That's a lot of rows to compare and the assert is wrong anyways!

Let's make this test assert on the thing we actually care about - the output price of the diamond!

## Instructions - Chispa

#### With Chispa
- [ ] Add these imports to the top of the cell, below the `%%ipytest -qq` line:  
`from chispa import assert_column_equality`  
`from pySpark.sql.functions import lit`
- [ ] Filter the dataframe for the unique id of the diamond we care about:  
`actual_df=actual_df.filter(actual_df.id == 'DI-26-null-price')`
- [ ] Create a new column in our dataframe that contains our expected price:  
`actual_df=actual_df.withColumn('expected_price', lit(2690.0))`
- [ ] Assert the value in the price column matches the value we want:  
`assert_column_equality(actual_df, 'price', 'expected_price')`


## Exercise - Chispa

In [ ]:
%%ipytest -qq
from pyspark.sql import SparkSession


def test_null_price_is_replaced_based_on_cut_clarity_and_color(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    actual_df = transform(diamonds_df)


.                                                                                            [100%]


### Instructions - Pandas
- [ ] import pandas:  
`import pandas as pd`
- [ ] Filter the dataframe for the unique id of the diamond we care about:  
  `actual_df=actual_df.filter(actual_df.id == 'DI-26-null-price')`
- [ ] Create your expected dataframe using Pandas:  
 `expected = pd.DataFrame(({'id': ["DI-26-null-price"], 'price':[2690.0] }))`
- [ ] Select the column you care about:  
  `actual_df=actual_df.select(['id', 'price'])
- [ ] Assert for dataframe equality using pandas:  
  `pd.testing.assert_frame_equal(actual_df, expected)`

## Exercise - Pandas

In [ ]:
%%ipytest -qq
from pyspark.sql import SparkSession


def test_null_price_is_replaced_based_on_cut_clarity_and_color(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    actual_df = transform(diamonds_df)


.                                                                                            [100%]


### Instructions - JSON properties

- [ ] Filter the dataframe for the unique id of the diamond we care about:  
  `actual_df=actual_df.filter(actual_df.id == 'DI-26-null-price')`
- [ ] Convert your dataframe to JSON:  
`actual_df_json = data_frame_to_json(actual_df)`
- [ ] Assert the price property of the first object matches your expected price:  
`assert actual_df_json[0]['price'] == 2690.0`

## Exercise - JSON properties

In [ ]:
%%ipytest -qq
from pyspark.sql import SparkSession


def test_null_price_is_replaced_based_on_cut_clarity_and_color(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    actual_df = transform(diamonds_df)



.                                                                                            [100%]


# Step 3: Reduce Duplicate Coverage and Fix the Bug

Right now, our test is running the entire transform function. Because there are multiple tests in `diamonds.json`, each test is running the same large block of code over and over again.

## STEP 3.A: Prep

### Instructions
Let's get ready to reduce the duplicate coverage.

#### 1. Put the transform function where you can work with it
- [ ] Run the test. It should be failing.
- [ ] Replace the call to the transform function with the body of that function.
- [ ] Change the last line of the function body to assign to `actual_df` instead of `df`
```
df = build_indep_vars(df, ['carat', 'clarity', 'color'],
                                      categorical_vars=['clarity', 'color'],
                                      keep_intermediate=False,
                                      summarizer=True)
```
becomes
```
actual_df = build_indep_vars(df, ['carat', 'clarity', 'color'],
                                      categorical_vars=['clarity', 'color'],
                                      keep_intermediate=False,
                                      summarizer=True)
```
- [ ] Change the first line of the function body to read from `diamonds_df` instead of `df`
```
 df = df.withColumn('lprice', log('price'))
```
becomes
```
 df = diamonds_df.withColumn('lprice', log('price'))
```
- [ ] Run the test. It should still be failing for the same reasons.

#### 2. Test FOR the bug
- [ ] Change your assert code so that it is testing _for_ the bug.
```
    actual_df=actual_df.withColumn('expected_price', lit(2690.0))
```
becomes
```
  actual_df=actual_df.withColumn('expected_price', lit(2460.0))
```
- [ ] Run the test. It should pass.

#### 3. Make the assert easier to work with
- [ ] Extract your assert code into a one-line helper function:
```
def assert_diamond_has_expected_price(actual_df):
    actual_df=actual_df.filter(actual_df.id == 'DI-26-null-price')
    actual_df=actual_df.withColumn('expected_price', lit(2460.0))
    assert_column_equality(actual_df, 'price', 'expected_price')
```
- [ ] Run the test. It should pass.

### Exercise

#### The Code

##### The `transform` Function

In [ ]:
from pyspark.sql import DataFrame, Window, Column
from pyspark.sql.functions import log, when, mean, col


def transform(df: DataFrame) -> DataFrame:
# The body of the transform function STARTS HERE
    df = df.withColumn('lprice', log('price'))  #<-- In the test, this line should be: df = diamonds_df.withColumn('lprice', log('price'))
    window = Window.partitionBy('cut', 'clarity').orderBy('price').rowsBetween(-3, 3)
    moving_avg = mean(df['price']).over(window)
    df = df.withColumn('moving_avg', moving_avg)

    df = df.withColumn('price', when(df.price.isNull(), df.moving_avg).otherwise(df.price))
    df = df[['id', 'carat', 'clarity', 'color', 'price']]
    df = build_indep_vars(df, ['carat', 'clarity', 'color'], #<-- In the test, this line should be: actual_df = build_indep_vars(df, ['carat', 'clarity', 'color']
                                      categorical_vars=['clarity', 'color'],
                                      keep_intermediate=False,
                                      summarizer=True)
# The body of the transform function ENDS HERE
    return df

#### The Test

In [ ]:
%%ipytest -qq
from pyspark.sql import SparkSession

def test_prep_for_linear_regression(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    actual_df = transform(diamonds_df) #<-- We will be replacing this line with the body of the transform function

    actual_df=actual_df.filter(actual_df.id == 'DI-26-null-price')
    actual_df=actual_df.withColumn('expected_price', lit(2690.0))
    assert_column_equality(actual_df, 'price', 'expected_price')

F                                                                                            [100%]
============================================= FAILURES =============================================
_________________________________ test_prep_for_linear_regression __________________________________

spark = <pyspark.sql.session.SparkSession object at 0x7fd3843987c0>

    def test_prep_for_linear_regression(spark: SparkSession):
        diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)
    
        actual_df = transform(diamonds_df)
        actual_df.show()
        actual_df=actual_df.filter(actual_df.id == 'DI-26-null-price')
        actual_df=actual_df.withColumn('expected_price', lit(2690.0))
>       assert_column_equality(actual_df, 'price', 'expected_price')

<ipython-input-90-613d013a3b70>:10: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

df = DataFrame[id: string, carat: double, clarity: string, col

#### The original test



In [ ]:
%%ipytest -qq
from pyspark.sql import SparkSession

def test_will_do_the_right_thing(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    actual_df = transform(diamonds_df)
    assert data_frame_to_json(actual_df) == expected_json("expected_correct.json")

## Step 3.B Squeeze the bottom!



### Instructions
**our bug**: Diamonds of the same cut and clarity are influencing the calculated price of diamonds with a different color. Only diamonds with the same cut, clarity and color should be influencing the calculated price for diamonds with a null price.

**Squeeze the bottom until you find the bug**
- [ ] Move your assert up one line at a time.  
- [ ] After each move, run your test.  
- [ ] If it fails, figure out why it's failing.(You may need to rename columns in the assert)
- [ ] If the test passes, the line wasn't important for the bug you wanted to catch. Delete it.
- [ ] Continue until you find the source of the bug

### Exercise

#### The Code

##### The `transform` Function

In [ ]:
from pyspark.sql import DataFrame, Window, Column
from pyspark.sql.functions import log, when, mean, col

def transform(df: DataFrame) -> DataFrame:

    df = df.withColumn('lprice', log('price'))
    window = Window.partitionBy('cut', 'clarity').orderBy('price').rowsBetween(-3, 3)
    moving_avg = mean(df['price']).over(window)
    df = df.withColumn('moving_avg', moving_avg)

    df = df.withColumn('price', when(df.price.isNull(), df.moving_avg).otherwise(df.price))
    df = df[['id', 'carat', 'clarity', 'color', 'price']]
    df = build_indep_vars(df, ['carat', 'clarity', 'color'],
                                      categorical_vars=['clarity', 'color'],
                                      keep_intermediate=False,
                                      summarizer=True)
    return df

#### The Test

In [ ]:
%%ipytest -qq
from pyspark.sql import SparkSession, DataFrame, Window, Column
from pyspark.sql.functions import lit, log, when, mean, col
from chispa import assert_column_equality

def assert_diamond_has_expected_price(actual_df):
    actual_df=actual_df.filter(actual_df.id == 'DI-26-null-price')
    actual_df=actual_df.withColumn('expected_price', lit(2460.0))
    assert_column_equality(actual_df, 'price', 'expected_price')

def test_prep_for_linear_regression(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    actual_df = diamonds_df.withColumn('lprice', log('price'))
    window = Window.partitionBy('cut', 'clarity').orderBy('price').rowsBetween(-3, 3)
    moving_avg = mean(actual_df['price']).over(window)
    actual_df = actual_df.withColumn('moving_avg', moving_avg)

    actual_df = actual_df.withColumn('price', replace_null(col('price'), col('moving_avg')))
    actual_df = actual_df[['id', 'carat', 'clarity', 'color', 'price']]
    actual_df = build_indep_vars(actual_df, ['carat', 'clarity', 'color'],
                                      categorical_vars=['clarity', 'color'],
                                      keep_intermediate=False,
                                      summarizer=True)

    assert_diamond_has_expected_price(actual_df)





.                                                                                            [100%]


#### The Original Test

In [ ]:
%%ipytest -qq
from pyspark.sql import SparkSession

def test_will_do_the_right_thing(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    actual_df = transform(diamonds_df)
    assert data_frame_to_json(actual_df) == expected_json("expected_correct.json")

## Step 3.C Let's fix the bug!


### Instructions

**Our Bug**: Diamonds of the same cut and clarity are influencing the calculated price of diamonds with a different color. Only diamonds with the same cut, clarity and color should be influencing the calculated price for diamonds with a null price.

**Our Desired Behaviour**:  
The input diamond with these properties:
- id: `"DI-26-null-price"`
- cut: `"Good"`
- color: `"D"`
- clarity: `"VVS1"`
- price: `null`

Should be output with a price of `2690.0` - the average price of the other diamonds with `cut="Good"`, `clarity="VVS1"` and `color="D"`

**Fix the bug**
#### Test for the behaviour you actually want
- [ ] Update your test so that it checks for the good behaviour.  
Replace the expected price on this line with `2690.0`:  
```
actual_df=actual_df.withColumn('expected_price', lit(2460.0))
```
- [ ] Run your test. It should fail with a `columnsNotEqualError`:  
```
E           chispa.column_comparer.ColumnsNotEqualError:
E           +------------+----------------+
E           | moving_avg | expected_price |
E           +------------+----------------+
E           |   2460.0   |     2690.0     |
E           +------------+----------------+
```

#### Fix the bug
- [ ] Fix the code _in your test_ so that the bug is gone. We need to add `'color'` to `'cut'` and `'clarity'` on this line:
```
window = Window.partitionBy('cut', 'clarity').orderBy('price').rowsBetween(-3, 3)
```
- [ ] Run your test. It should pass.

#### Encapsulate the code necessary for the behaviour
- [ ] The behaviour belongs to a group of lines working together. Extract them into a method.  
These lines can't be separated without changing the behaviour we are testing:  
```
    window = Window.partitionBy('cut', 'clarity', 'color').orderBy('price').rowsBetween(-3, 3)
    moving_avg = mean(df['price']).over(window)
    df = df.withColumn('moving_avg', moving_avg)
    df = df.withColumn('price', when(df.price.isNull(), df.moving_avg).otherwise(df.price))
```  
Use them to make a method:
```
def calculate_avg_price_for_similar_diamonds(df: DataFrame) -> DataFrame:
      window = Window.partitionBy('cut', 'clarity', 'color').orderBy('price').rowsBetween(-3, 3)
      moving_avg = mean(df['price']).over(window)
      df = df.withColumn('moving_avg', moving_avg)
      df = df.withColumn('price', when(df.price.isNull(), df.moving_avg).otherwise(df.price))
      return df
```

Replace those lines with a call to the new method in your test:  
```
actual_df = calculate_avg_price_for_similar_diamonds(actual_df)
```
- [ ] Run your test. It should pass.

#### Move the encapsulated behaviour to the actual code
- [ ] Move the new method out of your test and into the transform code.
- [ ] Replace the lines in your transform code with the new method call.
- [ ] We've changed the original code, so we need to check that everything still works the way we expect. Run the copy of your original large test. It should also pass.
- [ ] Run your test. It should pass.


### Exercise

#### The Code

##### The `transform` Function

In [ ]:
from pyspark.sql import DataFrame, Window, Column
from pyspark.sql.functions import log, when, mean, col


def transform(df: DataFrame) -> DataFrame:

    df = df.withColumn('lprice', log('price'))
    window = Window.partitionBy('cut', 'clarity').orderBy('price').rowsBetween(-3, 3)
    moving_avg = mean(df['price']).over(window)
    df = df.withColumn('moving_avg', moving_avg)

    df = df.withColumn('price', when(df.price.isNull(), df.moving_avg).otherwise(df.price))
    df = df[['id', 'carat', 'clarity', 'color', 'price']]
    df = build_indep_vars(df, ['carat', 'clarity', 'color'],
                                      categorical_vars=['clarity', 'color'],
                                      keep_intermediate=False,
                                      summarizer=True)
    return df

#### The Test

In [ ]:
%%ipytest -qq
from pyspark.sql import SparkSession, DataFrame, Window, Column
from pyspark.sql.functions import lit, log, when, mean, col
from chispa import assert_column_equality

def assert_diamond_has_expected_price(actual_df):
    actual_df=actual_df.filter(actual_df.id == 'DI-26-null-price')
    actual_df=actual_df.withColumn('expected_price', lit(2460.0))
    assert_column_equality(actual_df, 'price', 'expected_price')

def test_prep_for_linear_regression(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    df = diamonds_df.withColumn('lprice', log('price'))
    window = Window.partitionBy('cut', 'clarity').orderBy('price').rowsBetween(-3, 3)
    moving_avg = mean(df['price']).over(window)
    df = df.withColumn('moving_avg', moving_avg)
    df = df.withColumn('price', when(df.price.isNull(), df.moving_avg).otherwise(df.price))

    assert_diamond_has_expected_price(df)





.                                                                                            [100%]


#### The Original Test

In [ ]:
%%ipytest -qq
from pyspark.sql import SparkSession

def test_will_do_the_right_thing(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    actual_df = transform(diamonds_df)
    assert data_frame_to_json(actual_df) == expected_json("expected_correct.json")

# Step 4: Simplify the Top

## Instructions

We don't want our test to be reading in the entire diamonds.json file in order to test this single behaviour. We are going to simplify things so that we have the absolute minimum number of inputs we need to reproduce the behaviour.

- [ ] We can see that we only care about the moving_avg column, and not the price column. It should be safe to delete this line:
```
actual_df = diamonds_df.withColumn('lprice', log('price'))
```
- [ ] Run the test to make sure everything still works


### Exercise

#### The Code

##### The `transform` Function

In [ ]:
from pyspark.sql import DataFrame, Window, Column
from pyspark.sql.functions import log, when, mean, col

def calculate_avg_price_for_similar_diamonds(diamonds_df: DataFrame) -> DataFrame:
    window = Window.partitionBy('cut', 'clarity', 'color').orderBy('price').rowsBetween(-3, 3)
    moving_avg = mean(diamonds_df['price']).over(window)
    diamonds_df = diamonds_df.withColumn('moving_avg', moving_avg)
    return diamonds_df

def transform(df: DataFrame) -> DataFrame:

    df = df.withColumn('lprice', log('price'))
    df = calculate_avg_price_for_similar_diamonds(df)

    df_new = df.withColumn('price', when(df.price.isNull(), df.moving_avg).otherwise(df.price))
    df = df[['id', 'carat', 'clarity', 'color', 'price']]
    df = build_indep_vars(df, ['carat', 'clarity', 'color'],
                                      categorical_vars=['clarity', 'color'],
                                      keep_intermediate=False,
                                      summarizer=True)
    return df

#### The Test

In [ ]:
%%ipytest -qq
from pyspark.sql import SparkSession, DataFrame, Window, Column
from pyspark.sql.functions import lit, log, when, mean, col
from chispa import assert_column_equality

def assert_diamond_has_expected_price(actual_df):
    actual_df=actual_df.filter(actual_df.id == 'DI-26-null-price')
    actual_df=actual_df.withColumn('expected_price', lit(2460.0))
    assert_column_equality(actual_df, 'moving_avg', 'expected_price')

def test_prep_for_linear_regression(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    actual_df = diamonds_df.withColumn('lprice', log('price'))
    actual_df = calculate_avg_price_for_similar_diamonds(actual_df)

    assert_diamond_has_expected_price(actual_df)





.                                                                                            [100%]


#### The Original Test

In [ ]:
%%ipytest -qq
from pyspark.sql import SparkSession

def test_will_do_the_right_thing(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    actual_df = transform(diamonds_df)
    assert data_frame_to_json(actual_df) == expected_json("expected_correct.json")

# Step 5: Clean up!

### Exercise

#### The Code

##### The `transform` Function

In [ ]:
from pyspark.sql import DataFrame, Window, Column
from pyspark.sql.functions import log, when, mean, col

def calculate_avg_price_for_similar_diamonds(diamonds_df: DataFrame) -> DataFrame:
    window = Window.partitionBy('cut', 'clarity', 'color').orderBy('price').rowsBetween(-3, 3)
    moving_avg = mean(diamonds_df['price']).over(window)
    diamonds_df = diamonds_df.withColumn('moving_avg', moving_avg)
    return diamonds_df

def transform(df: DataFrame) -> DataFrame:

    df = df.withColumn('lprice', log('price'))
    df = calculate_avg_price_for_similar_diamonds(df)

    df_new = df.withColumn('price', when(df.price.isNull(), df.moving_avg).otherwise(df.price))
    df = df[['id', 'carat', 'clarity', 'color', 'price']]
    df = build_indep_vars(df, ['carat', 'clarity', 'color'],
                                      categorical_vars=['clarity', 'color'],
                                      keep_intermediate=False,
                                      summarizer=True)
    return df

#### The Test

In [ ]:
%%ipytest -qq
from pyspark.sql import SparkSession, DataFrame, Window, Column
from pyspark.sql.functions import lit, log, when, mean, col
from chispa import assert_column_equality

def assert_diamond_has_expected_price(actual_df):
    actual_df=actual_df.filter(actual_df.id == 'DI-26-null-price')
    actual_df=actual_df.withColumn('expected_price', lit(2460.0))
    assert_column_equality(actual_df, 'moving_avg', 'expected_price')

def test_prep_for_linear_regression(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    actual_df = diamonds_df.withColumn('lprice', log('price'))
    actual_df = calculate_avg_price_for_similar_diamonds(actual_df)

    assert_diamond_has_expected_price(actual_df)





.                                                                                            [100%]


#### The Original Test

In [ ]:
%%ipytest -qq
from pyspark.sql import SparkSession

def test_will_do_the_right_thing(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    actual_df = transform(diamonds_df)
    assert data_frame_to_json(actual_df) == expected_json("expected_correct.json")